In [34]:
import pandas as pd
import numpy as np
import networkx as nx

data_type = "SiouxFalls"
network_df = pd.read_csv("../{}/{}_net.txt".format(data_type, data_type), sep='\t', comment=';')
node_df = pd.read_csv("../{}/{}_node.txt".format(data_type, data_type), sep='\t', comment=';')
od_df = pd.read_csv("../{}/{}_od.csv".format(data_type, data_type))

network_df = network_df[['init_node', 'term_node', 'capacity', 'length', 'free_flow_time', 'b',
       'power', 'speed', 'toll', 'link_type']]
network_df[['init_node', 'term_node']] = network_df[['init_node', 'term_node']].astype(int)
node_df = node_df[['Node', 'X', 'Y']]

print("Number of nodes:", len(node_df))
print("Number of links:", len(network_df))
print("Number of od pairs:", len(od_df))

# Find unique origins and destinations
origins = od_df['O'].unique()
destins = od_df['D'].unique()

od = []
for od_pair_index in range(len(od_df)):
    i,j = od_df['O'].iloc[od_pair_index], od_df['D'].iloc[od_pair_index]
    od.append((i,j))

road_link = [(int(row['init_node']), int(row['term_node']), row['length']) for _, row in network_df.iterrows()]

arcs = [(int(row['init_node']), int(row['term_node'])) for _, row in network_df.iterrows()]

n_routes = 3

OD_route = {}



def generate_route_sets_link_elimination(graph, source, target, num_routes):
    route_sets = []
    route_dict = {}
    r_ix = 0
    for i in range(num_routes):
        path = nx.shortest_path(graph, source=source, target=target, weight='weight')
        path = [int(p) for p in path]
        if path not in route_sets:
            route_sets.append(path)
            route_dict[int(r_ix)] = list(path)
            r_ix += 1 
        
        if len(path) > 2:
            edge_to_remove = (path[1], path[2])
            original_weight = graph[edge_to_remove[0]][edge_to_remove[1]]['weight']
            graph[edge_to_remove[0]][edge_to_remove[1]]['weight'] = float('inf')
            
        else:
            break
        
    # Reset graph weights for future usage
    nx.set_edge_attributes(graph, original_weight, 'weight')
    
    return route_dict

# link penalty approach
def generate_route_sets_link_penalty(graph, source, target, num_routes, penalty_factor=1.5):
    route_sets = []
    route_dict = {}
    r_ix = 0
    for i in range(num_routes):
        path = nx.shortest_path(graph, source=source, target=target, weight='weight')
        path = [int(p) for p in path]
        if path not in route_sets:
            route_sets.append(path)
            route_dict[int(r_ix)] = list(path)
            r_ix += 1 
        
        for j in range(len(path) - 1):
            edge = (path[j], path[j+1])
            graph[edge[0]][edge[1]]['weight'] *= penalty_factor 
            
    # Reset graph weights for future usage
    nx.set_edge_attributes(graph, 1, 'weight')

    return route_dict


for (i,j) in od:
    G1 = nx.DiGraph()
    G1.add_weighted_edges_from(road_link)
    route_sets = generate_route_sets_link_penalty(G1, i, j, n_routes) 
    OD_route[(int(i),int(j))] = route_sets 


def is_continuous_subsequence(my_tuple, my_list):
    tuple_length = len(my_tuple)
    list_length = len(my_list)
    
    for i in range(0, list_length - tuple_length + 1):
        if tuple(my_list[i:i+tuple_length]) == my_tuple:
            return True
    return False




Number of nodes: 24
Number of links: 76
Number of od pairs: 528


In [35]:
import pickle
pd.to_pickle(OD_route, "../SiouxFalls/OD_route.pickle")